In [1]:
import pandas as pd
train_data = pd.read_csv(r"C:\Users\Suvir Gupta\PycharmProjects\Projects\word2vec\nueral net\data\labeledTrainData.tsv",
                         sep = "\t", header= (0))

In [2]:
from nltk.stem.wordnet import WordNetLemmatizer
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
def preprocessing(reviews):
## remove the Html Tags
    review_text = BeautifulSoup(reviews).get_text()
## Remove non letters
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
## set all words to lower digits
    review_text = review_text.lower().split()
## remove stopwords
    stopword = set(stopwords.words("english"))
    words = [w for w in review_text if w not in stopword]
### lemmatization of the words to reduce them to their standard foramat
    wordnet_lemmatizer = WordNetLemmatizer()
    words = [wordnet_lemmatizer.lemmatize(w) for w in words]
    return words

In [7]:
vocabulary= [preprocessing(review) for review in train_data["review"]]

C:\Users\Suvir Gupta\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\Suvir Gupta\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [16]:
vocabulary_flat = [word for wd_list in vocabulary for word in wd_list]

In [20]:
vocabulary_size = 50000

In [22]:
import collections
def build_dataset(words, n_words):
  """Process raw inputs into a dataset."""
  count = [['UNK', -1]]
  count.extend(collections.Counter(words).most_common(n_words - 1))
  dictionary = dict()
  for word, _ in count:
    dictionary[word] = len(dictionary)
  data = list()
  unk_count = 0
  for word in words:
    if word in dictionary:
      index = dictionary[word]
    else:
      index = 0  # dictionary['UNK']
      unk_count += 1
    data.append(index)
  count[0][1] = unk_count
  reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
  return data, count, dictionary, reversed_dictionary

In [23]:
data, count, dictionary, reverse_dictionary = build_dataset(vocabulary_flat,
                                                            vocabulary_size)
# del vocabulary  # Hint to reduce memory.
print('Most common words (+UNK)', count[:5])
print('Sample data', data[:10], [reverse_dictionary[i] for i in data[:10]])

Most common words (+UNK) [['UNK', 16469], ('movie', 51694), ('film', 47034), ('one', 27741), ('like', 20740)]
Sample data [408, 77, 141, 8138, 519, 2372, 123, 65, 871, 451] ['stuff', 'going', 'moment', 'mj', 'started', 'listening', 'music', 'watching', 'odd', 'documentary']


In [26]:
del vocabulary_flat

NameError: name 'vocabulary_flat' is not defined

In [27]:
data_index = 0
import numpy as np
import random
def generate_batch(batch_size, num_skips, skip_window):
  global data_index
  assert batch_size % num_skips == 0
  assert num_skips <= 2 * skip_window
  batch = np.ndarray(shape=(batch_size), dtype=np.int32)
  labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
  span = 2 * skip_window + 1  # [ skip_window target skip_window ]
  buffer = collections.deque(maxlen=span)
  if data_index + span > len(data):
    data_index = 0
  buffer.extend(data[data_index:data_index + span])
  data_index += span
  for i in range(batch_size // num_skips):
    target = skip_window  # target label at the center of the buffer
    targets_to_avoid = [skip_window]
    for j in range(num_skips):
      while target in targets_to_avoid:
        target = random.randint(0, span - 1)
      targets_to_avoid.append(target)
      batch[i * num_skips + j] = buffer[skip_window]
      labels[i * num_skips + j, 0] = buffer[target]
    if data_index == len(data):
      buffer[:] = data[:span]
      data_index = span
    else:
      buffer.append(data[data_index])
      data_index += 1
  # Backtrack a little bit to avoid skipping words in the end of a batch
  data_index = (data_index + len(data) - span) % len(data)
  return batch, labels

batch, labels = generate_batch(batch_size=8, num_skips=2, skip_window=1)
for i in range(8):
  print(batch[i], reverse_dictionary[batch[i]],
        '->', labels[i, 0], reverse_dictionary[labels[i, 0]])

77 going -> 408 stuff
77 going -> 141 moment
141 moment -> 8138 mj
141 moment -> 77 going
8138 mj -> 519 started
8138 mj -> 141 moment
519 started -> 8138 mj
519 started -> 2372 listening


In [30]:
import tensorflow as tf
import math
batch_size = 128
embedding_size = 128  # Dimension of the embedding vector.
skip_window = 1       # How many words to consider left and right.
num_skips = 2         # How many times to reuse an input to generate a label.

# We pick a random validation set to sample nearest neighbors. Here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent.
valid_size = 16     # Random set of words to evaluate similarity on.
valid_window = 100  # Only pick dev samples in the head of the distribution.
valid_examples = np.random.choice(valid_window, valid_size, replace=False)
num_sampled = 64    # Number of negative examples to sample.

graph = tf.Graph()

with graph.as_default():

  # Input data.
  train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
  train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
  valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

  # Ops and variables pinned to the CPU because of missing GPU implementation
  with tf.device('/cpu:0'):
    # Look up embeddings for inputs.
    embeddings = tf.Variable(
        tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
    embed = tf.nn.embedding_lookup(embeddings, train_inputs)

    # Construct the variables for the NCE loss
    nce_weights = tf.Variable(
        tf.truncated_normal([vocabulary_size, embedding_size],
                            stddev=1.0 / math.sqrt(embedding_size)))
    nce_biases = tf.Variable(tf.zeros([vocabulary_size]))

  # Compute the average NCE loss for the batch.
  # tf.nce_loss automatically draws a new sample of the negative labels each
  # time we evaluate the loss.
  loss = tf.reduce_mean(
      tf.nn.nce_loss(weights=nce_weights,
                     biases=nce_biases,
                     labels=train_labels,
                     inputs=embed,
                     num_sampled=num_sampled,
                     num_classes=vocabulary_size))

  # Construct the SGD optimizer using a learning rate of 1.0.
  optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)

  # Compute the cosine similarity between minibatch examples and all embeddings.
  norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
  normalized_embeddings = embeddings / norm
  valid_embeddings = tf.nn.embedding_lookup(
      normalized_embeddings, valid_dataset)
  similarity = tf.matmul(
      valid_embeddings, normalized_embeddings, transpose_b=True)

  # Add variable initializer.
  init = tf.global_variables_initializer()

In [33]:
# Step 5: Begin training.
num_steps = 100001

with tf.Session(graph=graph) as session:
  # We must initialize all variables before we use them.
  init.run()
  print('Initialized')

  average_loss = 0
  for step in range(num_steps):
    batch_inputs, batch_labels = generate_batch(
        batch_size, num_skips, skip_window)
    feed_dict = {train_inputs: batch_inputs, train_labels: batch_labels}

    # We perform one update step by evaluating the optimizer op (including it
    # in the list of returned values for session.run()
    _, loss_val = session.run([optimizer, loss], feed_dict=feed_dict)
    average_loss += loss_val

    if step % 2000 == 0:
      if step > 0:
        average_loss /= 2000
      # The average loss is an estimate of the loss over the last 2000 batches.
      print('Average loss at step ', step, ': ', average_loss)
      average_loss = 0

    # Note that this is expensive (~20% slowdown if computed every 500 steps)
    if step % 10000 == 0:
      sim = similarity.eval()
      for i in range(valid_size):
        valid_word = reverse_dictionary[valid_examples[i]]
        top_k = 8  # number of nearest neighbors
        nearest = (-sim[i, :]).argsort()[1:top_k + 1]
        log_str = 'Nearest to %s:' % valid_word
        for k in range(top_k):
          close_word = reverse_dictionary[nearest[k]]
          log_str = '%s %s,' % (log_str, close_word)
        print(log_str)
  final_embeddings = normalized_embeddings.eval()

Initialized
Average loss at step  0 :  269.802703857


Nearest to part: hack, cytown, unrewarding, dey, rubble, excised, charlton, schooling,
Nearest to pretty: terrace, gor, stephane, ing, resisted, incendiary, skogland, piglet,
Nearest to film: afgahnistan, ungrateful, pounded, hypnotizes, cortese, expelled, dispossessed, norfolk,
Nearest to however: unbeliever, phonograph, apotheosising, plodded, untalented, microfilmed, swedish, dietrichesque,
Nearest to know: lightyears, quantas, terorism, holiday, devouring, hirjee, graciela, klebold,
Nearest to life: micah, thall, blesses, bearable, summing, wave, obscura, fish,
Nearest to want: cipher, llkick, buttgereit, contraception, unstylized, azumi, fungus, olde,
Nearest to cast: geared, kaldwell, goofiness, gain, strained, misdemeanor, profess, hayter,
Nearest to around: badmitton, jivetalking, somberness, audio, certifiable, collegiate, salazar, relentlessly,
Nearest to scene: butterfield, mustachioed, hagelin, bolder, chan, subliminal, pirouette, doco,
Nearest to going: thembrians, monies,

Average loss at step  2000 :  123.72834768


Average loss at step  4000 :  56.1250099983


Average loss at step  6000 :  35.4260886552


...

Nearest to part: hack, one, reliance, gosha, charlton, similarly, jouissance, basically,
Nearest to pretty: really, cinematography, aside, incendiary, colony, gallico, terrace, inhabitant,
Nearest to film: movie, one, show, also, story, katana, flick, cordell,
Nearest to however: one, UNK, unbeliever, cordell, great, movie, terrible, langley,
Nearest to know: camazotz, think, get, say, holiday, booker, sediment, langley,
Nearest to life: story, world, discussing, people, remarkable, nixon, booker, time,
Nearest to want: must, cuties, risqu, ji, cipher, huge, sleeper, think,
Nearest to cast: actor, strained, kitty, also, goofiness, weaker, wildly, hayter,
Nearest to around: skillfully, mcelwee, uncovers, audio, relentlessly, simulate, magna, badmitton,
Nearest to scene: sequence, one, thing, kirby, also, claus, way, film,
Nearest to going: katana, would, go, chore, milanese, ashraf, zorro, lamb,
Nearest to fact: contrasting, reverse, much, booker, algie, radiator, stronger, spoil,
Neare

Average loss at step  42000 :  5.63245602727


Average loss at step  44000 :  5.5552023766


Average loss at step  46000 :  5.47577293563


TypeError: sequence index must be integer, not 'slice'